In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product, combinations

## data load

In [226]:
store_data = pd.read_csv('onigiri.csv', sep=',')
store_array = store_data.values

In [229]:
store_data

,a,b,c,d,e
0,0,1,1,1,0
1,1,1,0,0,0
2,1,0,0,1,1
3,0,1,1,1,0
4,0,1,0,1,0


In [4]:
def GTI(df, indexes): # Get True Indexes
    '''
    Input
        df: dataframe (row: transactions, col: items)
        index: column indexes num to be examined 
    Output
        list of boolen
    '''
    if type(indexes) == int:
        indexes = [indexes]
    elif type(indexes) == np.int64:
        indexes = [indexes]
        
    return df[:,indexes]==1

In [5]:
def support(array_2d,  indexes, m='num'):
    gti_b = GTI(array_2d, indexes)
    if gti_b.shape[1]==0:
        return 0
    b = np.all(gti_b, axis=1)
    if m =='num':
        return np.sum(b)
    elif m =='ratio':
        return np.sum(b)/len(b)
    elif m == 'bool':
        return b1

In [6]:
def confidence(array_2d, X_indexes, Y_indexes):
    sup_X = support(array_2d, X_indexes)
    X_Y_indexes = np.hstack([X_indexes, Y_indexes])
    return support(array_2d, X_Y_indexes)/sup_X
    

In [7]:
GTI(store_array,[1,2])

array([[ True,  True],
       [ True, False],
       [False, False],
       [ True,  True],
       [ True, False]])

In [8]:
support(store_array, np.array([1,2]), m='ratio')

0.4

In [9]:
const = [True, False]
threshold = 0.2
for args in product(const, repeat=5):
    #print("args")
    #print(args)
    #print("items")
    #print(store_array[:,args])
    #print("support")
    sup = support(store_array, args, m='ratio') 
    if sup >= threshold:
        print("args")
        print(args)
        print("items")
        print(store_array[:,args])
        print("support")       
        print(support(store_array, args, m='ratio'))
        print()

args
(True, True, False, False, False)
items
[[0 1]
 [1 1]
 [1 0]
 [0 1]
 [0 1]]
support
0.2

args
(True, False, False, True, True)
items
[[0 1 0]
 [1 0 0]
 [1 1 1]
 [0 1 0]
 [0 1 0]]
support
0.2

args
(True, False, False, True, False)
items
[[0 1]
 [1 0]
 [1 1]
 [0 1]
 [0 1]]
support
0.2

args
(True, False, False, False, True)
items
[[0 0]
 [1 0]
 [1 1]
 [0 0]
 [0 0]]
support
0.2

args
(True, False, False, False, False)
items
[[0]
 [1]
 [1]
 [0]
 [0]]
support
0.4

args
(False, True, True, True, False)
items
[[1 1 1]
 [1 0 0]
 [0 0 1]
 [1 1 1]
 [1 0 1]]
support
0.4

args
(False, True, True, False, False)
items
[[1 1]
 [1 0]
 [0 0]
 [1 1]
 [1 0]]
support
0.4

args
(False, True, False, True, False)
items
[[1 1]
 [1 0]
 [0 1]
 [1 1]
 [1 1]]
support
0.6

args
(False, True, False, False, False)
items
[[1]
 [1]
 [0]
 [1]
 [1]]
support
0.8

args
(False, False, True, True, False)
items
[[1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 1]]
support
0.4

args
(False, False, True, False, False)
items
[[1]
 [0]
 [0]

In [10]:
def getF1(array_2D, minsup):
    '''
    Description
        Output frequent itemset from original tables (k=1)
    Input
        array_2D -> row: transactions, col: items
    Output
        frequent items' indexes: list
    '''
    return np.array([[col] for col in range(array_2D.shape[1]) if support(array_2D,col, m='ratio') >= minsup])

In [11]:
def getFkPlusOne(array_2D, indexes, minsup):
    '''
    Description
        Output frequent itemset from original tables (k=1)
    Input
        array_2D -> row: transactions, col: items
        indexes -> list of tuples [(),(),...()] 
    '''
    
    return np.array([col for col in indexes if support(array_2D,col, m='ratio') >= minsup])

In [12]:
F1 = getF1(store_array, minsup=0.4)

In [13]:
F1

array([[0],
       [1],
       [2],
       [3]])

In [14]:
def getCkPlusOne(prevCandidate, k):
    '''
    input
    prevCandidate: [(),(),...,()] # list of tuples
    k: length of next Candidate
    Output
      nextCandidate:[(),(),...,()] # list of tuples
     '''
    
    assert np.all(np.array([len(x) for x in prevCandidate])== k-1)
    assert k >1
    items = list(np.unique(np.array(prevCandidate).flatten())) 
    tmp_candidates = [x for x in combinations(items, k)]
    if k ==2:
        print("k=2, all combinations are output without filtering")
        return np.array(tmp_candidates)
        
    candidates = [
        candidate for candidate in tmp_candidates
        if all(
            x in prevCandidate
            for x in combinations(candidate, k - 1))
    ]
       
    return np.array(candidates)

In [15]:
C2 = getCkPlusOne(F1, 2)

k=2, all combinations are output without filtering


In [16]:
C2

array([[0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3],
       [2, 3]])

In [17]:
getFkPlusOne(store_array, C2, minsup=0.4)

array([[1, 2],
       [1, 3],
       [2, 3]])

In [18]:
def isEmpty(F):
    if len(F) < 1:
        return True
    else:
        return False

## extraction of Frequent Items

- k =1
- データベースを数え上げて$F_1$を生成
  - ループ先頭
  - $F_k$から$C_{k+1}$を生成
  - $C_{k+1}$中の集合が実際に頻出かどうかを、データベースを数え上げて $F_{k+1}$を生成
  - $F_{k+1}$が空ならばループを終了
  - $k= k+1;$ ループ先頭に戻る
- 出力: $F1, F2, ..., F_{k+1}$

In [19]:
db = store_array
minsum = 0.4
k = 1
F_list = []
F_now = getF1(store_array, minsum)
F_list.append(F_now)
while(True):
    print(k)
    print("F_now")
    print(F_now)
    print("calc C_k+1: k is set to %d" %  (k+1))
    C_next = getCkPlusOne(F_now, k+1)
    print("C_next")
    print(C_next)
    F_next = getFkPlusOne(store_array, C_next, minsum)
    if isEmpty(F_next):
        break
    k += 1
    F_now = F_next
    F_list.append(F_now)
    
del k

1
F_now
[[0]
 [1]
 [2]
 [3]]
calc C_k+1: k is set to 2
k=2, all combinations are output without filtering
C_next
[[0 1]
 [0 2]
 [0 3]
 [1 2]
 [1 3]
 [2 3]]
2
F_now
[[1 2]
 [1 3]
 [2 3]]
calc C_k+1: k is set to 3
C_next
[[1 2 3]]
3
F_now
[[1 2 3]]
calc C_k+1: k is set to 4
C_next
[]


Frequent itemの格納リスト(F_list)
リストの中に、k個の頻出アイテムリストが入っている（リストのリスト）

**k=1が0番目のリストに入っているためインデックスの時は注意**

各リストの中は2次元のnp.ndarray形式で入っている。
2D配列中の値はデータ中の列インデックス番号。
k=1のときは一つの列に対するfrequent item。
k=2のときは二つの列の組み合わせに対するfrequent item。
frequent itemは複数あるので2次元の配列にしている。


例

In [XX]: F_list

Out[XX]:  [
                array([[0],[1],[2],[3]]),  # k =1
                
                array([[1,2], [1,3], [2,3]]),  # k = 2
                
                array([[1,2,3]]),　# k = 3
                
                ...
                
                ]
                

In [20]:
F_list

[array([[0],
        [1],
        [2],
        [3]]), array([[1, 2],
        [1, 3],
        [2, 3]]), array([[1, 2, 3]])]

## 相関ルールの抽出

### 相関ルールのフォーマット

- conf_list: k個の相関ルールを格納するリスト
  - 0個めのインデックスにk=1がくるため、インデックスに注意

- **k=1のルールはない（「AならばB」のように最低2つのインデックスを使うので）**
  - ダミーとしてNoneを格納する
  
 conf_list の各要素の説明、
 [(array([1,2]), array([3])), ...]


- list(tuple(array, array)))の形
- tuple中の2つのarrayがantecedentとconsequent
- それをtupleでパッケージングしている。
- さらにそれが複数ありリストで格納している
 
 


In [111]:
conf_list = []
minconf = 0.7


'''
F_list =

    [array([[0],[1],[2],[3]]),          # k=1
      array([[1, 2],[1, 3],[2, 3]]),    # k=2
      array([[1, 2, 3]])],                     # k=3
      ...
      ]
'''
    
    
for F in F_list: # F_listの中の特定のkに対するfrequent items
    k = len(F[0])
    if k == 1:
        conf_list.append([None])
        pass
    elif k == 2:
        conf_list_k = []
        for f_2 in F:
            A = f_2[0]
            B = f_2[1]
            conf_AB = confidence(store_array, A, B)
            if conf_AB >= minconf:
                conf_list_k.append((np.array([A]),np.array([B])))
            conf_BA = confidence(store_array, B, A)
            if conf_BA >= minconf:
                conf_list_k.append((np.array([B]),np.array([A])))
        conf_list.append(conf_list_k)
    
    elif k >= 3:
        conf_list_k = []
        for f_k in F:    # 特定のkに対する一つ一つのfrequent item
           
            
            '''example
            k=3
            
            j=1
            f_k: [1,2,3]
            conf: ([1,2],[3]), ([1,3],[2]),([2,3],[1]) # j = 1
            f_kからj=1個consequentを作りだしてconfidenceを求める
            confidenceがminconf以上のものをconf_listに格納
            格納するものを
            ([1,2],[3]),([2,3],[1])とする(※とする)。　([1,3],[2])はminconf未満
            
            j = 2
            f_k: [1,2,3]
            conf: ([1],[2,3]),([2],[1,3]),([3],[1,2])
            リストアップしたconfの全てに対してconfidenceを計算すると時間がかかるので、
            効率的に解く。
            
            例えば([1],[2,3])のconfidenceを計算する必要があるのか？については
            consequentから一つをantecedentに戻したもの全てが※に含まれる必要がある
            
            [1],[2,3]からは
            [1,2],[3]　および [1,3],[2]が得られる。conf_listに格納した値は
            [1,2],[3] および　[2,3],[1]　であるので条件を満たさない
            故に[1],[2,3]はconfidenceを計算する必要がない。
            
            [2],[1,3]からは
            [1,2],[3]　および [2,3],[1]が得られる。conf_listに格納した値は
            [1,2],[3] および　[2,3],[1]　であるので条件を満たす
            故に[2],[1,3]はconfidenceを上回る可能性があるので計算する必要がある。
            
            上記の探索は結局 対象としている相関ルールのantecedentが前回の※の全ての
            antecedentに含まれていることに帰着できる。
            
            '''
            
            set_all = set(f_k)
            j= 1 # k個のantecedentからj個をconsequentに移動させてconfidenceを算出する
            
            # 新しい相関ルール候補の作成
            array_antecedent = np.array(list(combinations(f_k, k-1 )))
            array_consequent = np.array([tuple(set_all - set(c))  for c in array_antecedent])
            conf = np.array([confidence(store_array, ant, con) for ant, con in zip(array_antecedent, array_consequent)])
            
            isHigher = conf > minconf
            array_antecedent_filtered_by_conf = array_antecedent[isHigher]
            array_consequent_filtered_by_conf = array_consequent[isHigher]
            
            conf_list_k.append([(a,c) for a,c in zip(array_antecedent_filtered_by_conf, array_consequent_filtered_by_conf)])
         
           
           
            while(j < k-1):
                array_antecedent_new = np.array(list(combinations(f_k, k-(j+1) )))
                # filter antecedent by previous conf
                _res = [np.all([set(i) <= set(s) for s in array_antecedent_filtered_by_conf]) for i in array_antecedent_new]
                
                _res = isCalcConfNeeded(array_antecedent_filtered_by_conf, array_antecedent_new, set_all)
                
                array_antecedent_filtered_by_prev = array_antecedent_new[_res]
                array_consequent_filtered_by_prev = np.array([tuple(set_all - set(c))  for c in array_antecedent_filtered_by_prev])
                conf = np.array([confidence(store_array, ant, con) for ant, con in zip(array_antecedent_filtered_by_prev, 
                                                                                                                          array_consequent_filtered_by_prev)])
                isHigher = conf > minconf
                array_antecedent_filtered_by_prev_and_conf = array_antecedent_filtered_by_prev[isHigher]
                array_consequent_filtered_by_prev_and_conf = array_consequent_filtered_by_prev[isHigher]
                
                conf_list_k.append([(a,c) for a,c in zip(array_antecedent_filtered_by_prev_and_conf, 
                                                                                     array_consequent_filtered_by_prev_and_conf)])
                
                j += 1
        
        conf_list.append(conf_list_k)
            
            

In [109]:
j

1

In [110]:
conf_list

[[None],
 [(array([2]), array([1])),
  (array([1]), array([3])),
  (array([3]), array([1])),
  (array([2]), array([3]))],
 [[(array([1, 2]), array([3])), (array([2, 3]), array([1]))]]]

In [23]:
array_antecedent_filtered_by_conf

array([[1, 2],
       [1, 3],
       [2, 3]])

In [24]:
array_antecedent_new

array([[1],
       [2],
       [3]])

In [96]:
def isCalcConfNeeded(array_prev_ant, array_ant, set_f):
    
        '''
        Input 
            array_prev_ant: array([1,2], [1,3], [2,3])
            array_ant: array([1],[2],[3])
            set_f: [1,2,3]
            
        Var
            array_prev_con: array([3],[2],[1])
            array_con: array([2,3,[1,3],[1,2])
        
        '''
        array_prev_con  = np.array([tuple(set_f - set(c))  for c in array_prev_ant])
        array_con  = np.array([tuple(set_f - set(c))  for c in array_ant])
        
        out = []
        
        for a,c in zip(array_ant, array_con):
            out_inner = []
            for i in range(len(c)):
                array_ant_candidate = a
                cand = c[i]
                array_candidate_ant = np.append(a, cand)
                array_candidate_con = np.delete(c, i)
                
                res = np.any([set(array_candidate_ant) <= set(i)  for i in array_prev_ant])
                out_inner.append(res)
            if np.all(out_inner):
                out.append(True)
            else:
                out.append(False)
            
        return out
                
                
                
            
            
            


In [113]:
store_data = pd.read_csv("store_data.csv", header=None)

In [114]:
store_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
df = store_data.iloc[0:4,0:3]

In [163]:
df

,0,1,2
0,shrimp,almonds,avocado
1,burgers,meatballs,eggs
2,chutney,NaN,NaN
3,turkey,avocado,NaN


In [177]:
store_data = store_data.fillna("none")

In [165]:
from sklearn.preprocessing import MultiLabelBinarizer

In [178]:
mlb = MultiLabelBinarizer()
result = mlb.fit_transform(store_data.values)
store_data_trans = pd.DataFrame(result, columns = mlb.classes_).drop('none', axis=1)

In [179]:
store_data_trans

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
db = store_data_trans.values
minsum = 0.4

def frequent(db, minsum):
    
    k = 1
    F_list = []
    F_now = getF1(db, minsum)
    F_list.append(F_now)
    while(True):
        print(k)
        #print("F_now")
        #print(F_now)
        #print("calc C_k+1: k is set to %d" %  (k+1))
        C_next = getCkPlusOne(F_now, k+1)
        #print("C_next")
        #print(C_next)
        F_next = getFkPlusOne(db, C_next, minsum)
        if isEmpty(F_next):
            break
        k += 1
        F_now = F_next
        F_list.append(F_now)
        
    return F_list

F_list = frequent(store_data.values, minsum)
print(F_list)

1
k=2, all combinations are output without filtering
2
3
[array([[0],
       [1],
       [2],
       [3]]), array([[1, 2],
       [1, 3],
       [2, 3]]), array([[1, 2, 3]])]


In [237]:
conf_list = []
minconf = 0.7


'''
F_list =

    [array([[0],[1],[2],[3]]),          # k=1
      array([[1, 2],[1, 3],[2, 3]]),    # k=2
      array([[1, 2, 3]])],                     # k=3
      ...
      ]
'''
    

def find_rules(db, F_list, minconf) :
    
    for F in F_list: # F_listの中の特定のkに対するfrequent items
        k = len(F[0])
        print("k: ", k)
        if k == 1:
            conf_list.append([None])
            pass
        elif k == 2:
            conf_list_k = []
            for f_2 in F:
                A = f_2[0]
                B = f_2[1]
                conf_AB = confidence(db, A, B)
                if conf_AB >= minconf:
                    conf_list_k.append((np.array([A]),np.array([B])))
                conf_BA = confidence(db, B, A)
                if conf_BA >= minconf:
                    conf_list_k.append((np.array([B]),np.array([A])))
            conf_list.append(conf_list_k)
        
        elif k >= 3:
            conf_list_k = []
            for f_k in F:    # 特定のkに対する一つ一つのfrequent item
               
                
                '''example
                k=3
                
                j=1
                f_k: [1,2,3]
                conf: ([1,2],[3]), ([1,3],[2]),([2,3],[1]) # j = 1
                f_kからj=1個consequentを作りだしてconfidenceを求める
                confidenceがminconf以上のものをconf_listに格納
                格納するものを
                ([1,2],[3]),([2,3],[1])とする(※とする)。　([1,3],[2])はminconf未満
                
                j = 2
                f_k: [1,2,3]
                conf: ([1],[2,3]),([2],[1,3]),([3],[1,2])
                リストアップしたconfの全てに対してconfidenceを計算すると時間がかかるので、
                効率的に解く。
                
                例えば([1],[2,3])のconfidenceを計算する必要があるのか？については
                consequentから一つをantecedentに戻したもの全てが※に含まれる必要がある
                
                [1],[2,3]からは
                [1,2],[3]　および [1,3],[2]が得られる。conf_listに格納した値は
                [1,2],[3] および　[2,3],[1]　であるので条件を満たさない
                故に[1],[2,3]はconfidenceを計算する必要がない。
                
                [2],[1,3]からは
                [1,2],[3]　および [2,3],[1]が得られる。conf_listに格納した値は
                [1,2],[3] および　[2,3],[1]　であるので条件を満たす
                故に[2],[1,3]はconfidenceを上回る可能性があるので計算する必要がある。
                
                上記の探索は結局 対象としている相関ルールのantecedentが前回の※の全ての
                antecedentに含まれていることに帰着できる。
                
                '''
                
                set_all = set(f_k)
                j= 1 # k個のantecedentからj個をconsequentに移動させてconfidenceを算出する
                
                # 新しい相関ルール候補の作成
                array_antecedent = np.array(list(combinations(f_k, k-1 )))
                array_consequent = np.array([tuple(set_all - set(c))  for c in array_antecedent])
                conf = np.array([confidence(db, ant, con) for ant, con in zip(array_antecedent, array_consequent)])
                
                isHigher = conf > minconf
                if sum(isHigher) > 0:
                    array_antecedent_filtered_by_conf = array_antecedent[isHigher]
                    array_consequent_filtered_by_conf = array_consequent[isHigher]
                    
                    conf_list_k.append([(a,c) for a,c in zip(array_antecedent_filtered_by_conf, array_consequent_filtered_by_conf)])
             
               
               
                    while(j < k-1):
                        array_antecedent_new = np.array(list(combinations(f_k, k-(j+1) )))
                        # filter antecedent by previous conf
                        
                        _res = isCalcConfNeeded(array_antecedent_filtered_by_conf, array_antecedent_new, set_all)
                        
                        array_antecedent_filtered_by_prev = array_antecedent_new[_res]
                        array_consequent_filtered_by_prev = np.array([tuple(set_all - set(c))  for c in array_antecedent_filtered_by_prev])
                        conf = np.array([confidence(db, ant, con) for ant, con in zip(array_antecedent_filtered_by_prev, 
                                                                                                                                  array_consequent_filtered_by_prev)])
                        isHigher = conf > minconf
                        if sum(isHigher) > 0:
                            array_antecedent_filtered_by_prev_and_conf = array_antecedent_filtered_by_prev[isHigher]
                            array_consequent_filtered_by_prev_and_conf = array_consequent_filtered_by_prev[isHigher]
                            
                            conf_list_k.append([(a,c) for a,c in zip(array_antecedent_filtered_by_prev_and_conf, 
                                                                                                 array_consequent_filtered_by_prev_and_conf)])
                        
                        j += 1
            
            conf_list.append(conf_list_k)
    
    return conf_list
                
find_rules(store_data. values, F_list, minconf)            

k:  1
k:  2
k:  3


[[None],
 [(array([2]), array([1])),
  (array([1]), array([3])),
  (array([3]), array([1])),
  (array([2]), array([3]))],
 [[(array([1, 2]), array([3])), (array([2, 3]), array([1]))],
  [(array([2]), array([1, 3]))]]]

In [238]:
# mnist data

In [243]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

# The digits dataset
digits = datasets.load_digits()

n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)


In [246]:
X_test

array([[ 0.,  0.,  1., ..., 12.,  1.,  0.],
       [ 0.,  0.,  6., ...,  6.,  0.,  0.],
       [ 0.,  0.,  0., ...,  2.,  0.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])